# ETF Trading Pipeline - Full End-to-End

Single notebook that runs the entire pipeline:

| Section | Description | Cache |
|---------|-------------|-------|
| 0 | Setup & Configuration | - |
| 1 | Universe Discovery (~5,000 ETFs from NASDAQ + curated list) | `data/raw/etf_universe_qualified.csv` |
| 2 | Historical Data Collection (IB + yfinance, resumable) | `data/processed/etf_prices_ib.parquet` |
| 3 | Factor Scoring (Mom/Qual/Val/Vol) — leveraged filtered here | `data/factor_scores_latest.parquet` |
| 4 | Portfolio Construction & Optimization | `~/trading/live_portfolio/target_portfolio_latest.csv` |
| 5 | Backtesting & Performance Analysis | - |
| 6 | Trade Recommendations ($70k reserve) | `~/trading/live_portfolio/trade_plan.csv` |
| 7 | Edit Instructions & Claude Interpretation | - |
| 8 | Trade Execution (IB Gateway) | `~/trading/live_portfolio/execution_log.csv` |

Each section is independently runnable if cached data from prior steps exists.

---

## Section 0: Setup & Configuration

In [ ]:
import csv
import json
import os
import shutil
import sys
import warnings
from datetime import datetime
from pathlib import Path

import nest_asyncio
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

warnings.filterwarnings("ignore")
nest_asyncio.apply()

# ── Paths ──────────────────────────────────────────────
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

DATA_DIR = Path.home() / "trade_data" / "ETFTrader"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
IB_CACHE_DIR = DATA_DIR / "ib_historical"
TRADING_DIR = Path.home() / "trading"
LIVE_DIR = TRADING_DIR / "live_portfolio"
ARCHIVE_DIR = LIVE_DIR / "trade_plan_archive"
TRADE_PLAN_FILE = LIVE_DIR / "trade_plan.csv"

for d in [PROCESSED_DIR, LIVE_DIR, ARCHIVE_DIR, IB_CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ── IB Settings ────────────────────────────────────────
IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 5

# ── Strategy Parameters ────────────────────────────────
FACTOR_WEIGHTS = {
    "momentum": 0.35,
    "quality": 0.30,
    "value": 0.15,
    "volatility": 0.20,
}
NUM_POSITIONS = 20
OPTIMIZER_TYPE = "rankbased"  # mvo | rankbased | simple | minvar
CASH_RESERVE = 70_000
TRAILING_STOP_PCT = 10
STOP_LOSS_PCT = 0.12
DRIFT_THRESHOLD = 0.05

print(f"Project:       {PROJECT_ROOT}")
print(f"Data:          {DATA_DIR}")
print(f"Trading:       {TRADING_DIR}")
print(f"Optimizer:     {OPTIMIZER_TYPE}")
print(f"Positions:     {NUM_POSITIONS}")
print(f"Cash Reserve:  ${CASH_RESERVE:,}")
print(f"Trailing Stop: {TRAILING_STOP_PCT}%")


---
## Section 1: Universe Discovery

In [ ]:
from data_collection.comprehensive_etf_list import (
    COMPREHENSIVE_ETF_UNIVERSE,
    get_all_tickers,
    load_full_universe,
)
from data_collection.etf_filters import filter_leveraged_etfs, LEVERAGED_ETFS

# ── Load the FULL US ETF universe ──
# Merges our curated ~790 categorized tickers with the NASDAQ trader file (~4,900+)
all_tickers, categories = load_full_universe()

curated_count = sum(1 for t in all_tickers if categories[t] != "Uncategorized")
uncategorized_count = len(all_tickers) - curated_count
leveraged_count = sum(1 for t in all_tickers if t in LEVERAGED_ETFS)

print(f"Full universe: {len(all_tickers)} ETFs")
print(f"  Categorized (curated):  {curated_count}")
print(f"  Uncategorized (NASDAQ): {uncategorized_count}")
print(f"  Leveraged/inverse:      {leveraged_count} (kept for data collection)")

# Check for cached qualified universe
qualified_path = RAW_DIR / "etf_universe_qualified.csv"
if qualified_path.exists():
    universe_df = pd.read_csv(qualified_path)
    # Check if the cached universe is smaller than our full list (stale cache)
    if len(universe_df) < len(all_tickers) * 0.8:
        print(f"\nCached qualified universe ({len(universe_df)}) is much smaller "
              f"than full universe ({len(all_tickers)}). Rebuilding...")
        universe_df = pd.DataFrame({
            "ticker": all_tickers,
            "category": [categories.get(t, "Uncategorized") for t in all_tickers],
        })
    else:
        print(f"\nLoaded cached qualified universe: {len(universe_df)} tickers")
else:
    universe_df = pd.DataFrame({
        "ticker": all_tickers,
        "category": [categories.get(t, "Uncategorized") for t in all_tickers],
    })

display(
    Markdown(f"### Universe: {len(universe_df)} ETFs across "
             f"{universe_df['category'].nunique()} categories")
)
display(universe_df.groupby("category").size().sort_values(ascending=False).head(15).to_frame("count"))

In [ ]:
# Optional: qualify contracts on IB to prune non-existent tickers
# Set QUALIFY_ON_IB = True to run (takes ~2 min for 2500 tickers)
QUALIFY_ON_IB = False

if QUALIFY_ON_IB:
    from ib_insync import IB, Stock

    ib = IB()
    try:
        ib.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID, readonly=True, timeout=10)
        print(f"Connected: {ib.managedAccounts()[0]}")

        qualified, failed = [], []
        tickers_to_check = universe_df["ticker"].tolist()

        for i in range(0, len(tickers_to_check), 50):
            batch = tickers_to_check[i : i + 50]
            for t in batch:
                c = Stock(t, "SMART", "USD")
                try:
                    ib.qualifyContracts(c)
                    if c.conId:
                        qualified.append(t)
                    else:
                        failed.append(t)
                except Exception:
                    failed.append(t)
            ib.sleep(0.5)
            print(f"\r  {min(i + 50, len(tickers_to_check))}/{len(tickers_to_check)}", end="")

        universe_df = pd.DataFrame(
            {"ticker": qualified, "category": [categories.get(t, "Unknown") for t in qualified]}
        )
        universe_df.to_csv(qualified_path, index=False)
        print(f"\nQualified: {len(qualified)}, Failed: {len(failed)}")
        ib.disconnect()

    except Exception as e:
        print(f"IB not available: {e}")
        print("Using unqualified universe.")

---
## Section 2: Historical Data Collection

IB Gateway primary, yfinance fallback. Per-ticker parquet caching with smart resume:

- **CURRENT** tickers (cached & up-to-date): skipped, no IB request
- **STALE** tickers (cached but old): incremental update — fetches only the gap (e.g. last 30 days)
- **MISSING** tickers (no cache): full 5Y download

Set `RUN_COLLECTION = True` to connect to IB. Every ticker logs its action clearly.

In [ ]:
# Check for cached price data
prices_ib_path = PROCESSED_DIR / "etf_prices_ib.parquet"
prices_yf_path = PROCESSED_DIR / "etf_prices_filtered.parquet"

prices = None

if prices_ib_path.exists():
    prices = pd.read_parquet(prices_ib_path)
    print(f"Loaded IB prices: {prices.shape[1]} tickers x {prices.shape[0]} days")
    print(f"  Range: {prices.index[0].date()} to {prices.index[-1].date()}")
elif prices_yf_path.exists():
    prices = pd.read_parquet(prices_yf_path)
    print(f"Loaded yfinance prices: {prices.shape[1]} tickers x {prices.shape[0]} days")
else:
    # Try to build from individual parquets (e.g. after a partial IB download)
    cached_parquets = list(IB_CACHE_DIR.glob("*.parquet"))
    cached_parquets = [f for f in cached_parquets if f.stem != "manifest"]
    if len(cached_parquets) > 50:
        print(f"No consolidated file, but found {len(cached_parquets)} cached parquets. Building matrix...")
        from data_collection.ib_data_collector import IBDataCollector
        # Build without IB connection — just reads parquets
        class _DummyIB:
            pass
        _collector = IBDataCollector(ib=_DummyIB(), cache_dir=str(IB_CACHE_DIR))
        prices = _collector.build_price_matrix()
    else:
        print("No cached prices. Set COLLECT_DATA = True below to download.")

In [ ]:
# ============================================================
# Set to True to connect to IB and collect/update data.
# The collector is SMART — it will:
#   - SKIP tickers that are already up-to-date
#   - INCREMENT stale tickers (fetch only the gap, e.g. last 30 days)
#   - FULL DOWNLOAD only tickers with no cached data
# ============================================================
RUN_COLLECTION = True
SCAN_IB = False          # Also discover ETFs via IB scanner
# ============================================================

# Force reload of the module (in case kernel has old version cached)
import importlib
import data_collection.ib_data_collector as _ib_mod
importlib.reload(_ib_mod)
from data_collection.ib_data_collector import IBDataCollector, collect_with_fallback

tickers = universe_df["ticker"].tolist()

# Show what we already have cached
cached_parquets = [f.stem for f in IB_CACHE_DIR.glob("*.parquet") if f.stem != "manifest"]
print(f"Universe: {len(tickers)} tickers")
print(f"Already cached: {len(cached_parquets)} parquets in {IB_CACHE_DIR}")
print(f"Not yet cached: ~{len(set(tickers) - set(cached_parquets))} tickers\n")

# Optionally expand universe via IB scanner
if SCAN_IB:
    try:
        from ib_insync import IB
        ib_scan = IB()
        ib_scan.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID + 1, readonly=True, timeout=10)
        collector_scan = IBDataCollector(ib=ib_scan, cache_dir=str(IB_CACHE_DIR))
        scanned = collector_scan.scan_ib_etf_universe()
        if scanned:
            before = len(tickers)
            tickers = sorted(set(tickers) | set(scanned))
            print(f"IB scanner expanded universe: {before} -> {len(tickers)} tickers")
        ib_scan.disconnect()
    except Exception as e:
        print(f"IB scanner unavailable: {e}")

if RUN_COLLECTION:
    try:
        from ib_insync import IB

        ib_data = IB()
        ib_data.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID, readonly=True, timeout=10)
        print(f"Connected to IB: {ib_data.managedAccounts()[0]}\n")

        collector = IBDataCollector(
            ib=ib_data,
            cache_dir=str(IB_CACHE_DIR),
            rate_limit_interval=12.0,
            duration="5 Y",
        )

        # Single smart call: skips current, increments stale, downloads new
        prices, results = collector.collect_universe(tickers)

        ib_data.disconnect()

    except Exception as e:
        print(f"\nIB failed: {e}")
        if prices is None:
            print("Falling back to yfinance for top 500 tickers...")
            import yfinance as yf
            yf_tickers = [t for t in tickers if t not in cached_parquets][:500]
            data = yf.download(" ".join(yf_tickers), period="5y", auto_adjust=True)
            prices = data["Close"].dropna(axis=1, how="all")
            prices.to_parquet(PROCESSED_DIR / "etf_prices_ib.parquet")
        else:
            print(f"Using existing cached prices ({prices.shape[1]} tickers).")

In [ ]:
if prices is not None:
    # Data quality filter
    missing_pct = prices.isnull().sum() / len(prices) * 100
    adequate = (prices.count() >= 252) & (missing_pct < 10)
    before = len(prices.columns)
    prices = prices.loc[:, adequate]
    prices = prices.ffill().bfill()  # Forward-fill gaps

    print(f"Quality filter: {before} -> {len(prices.columns)} tickers")
    print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")
    print(f"Trading days: {len(prices)}")
else:
    print("No price data available. Cannot proceed to factor scoring.")

---
## Section 3: Factor Scoring

4-factor model: Momentum (35%), Quality (30%), Value (15%), Volatility (20%)

**Note:** Leveraged/inverse ETFs are filtered out HERE (not in data collection).
All price data is kept for future leveraged strategies.

In [ ]:
from factors import (
    FactorIntegrator,
    MomentumFactor,
    QualityFactor,
    SimplifiedValueFactor,
    VolatilityFactor,
)

factor_scores_path = DATA_DIR / "factor_scores_latest.parquet"

# ── Filter leveraged/inverse ETFs for the basic model ──
# The full price data (including leveraged) is preserved for other strategies
if prices is not None:
    all_price_tickers = prices.columns.tolist()
    basic_tickers = filter_leveraged_etfs(all_price_tickers)
    prices_basic = prices[basic_tickers]
    print(f"Basic model universe: {len(basic_tickers)} tickers "
          f"({len(all_price_tickers) - len(basic_tickers)} leveraged/inverse excluded)")
else:
    prices_basic = prices

# Check if recent scores exist (< 7 days old)
recalculate = True
if factor_scores_path.exists():
    age_days = (datetime.now().timestamp() - factor_scores_path.stat().st_mtime) / 86400
    if age_days < 7:
        print(f"Factor scores are {age_days:.1f} days old. Loading cached.")
        factor_df = pd.read_parquet(factor_scores_path)
        combined_scores = factor_df.iloc[:, 0].sort_values(ascending=False)
        recalculate = False
    else:
        print(f"Factor scores are {age_days:.1f} days old. Recalculating.")

if recalculate and prices_basic is not None:
    print("Calculating factor scores...")

    momentum_scores = MomentumFactor(lookback=252, skip_recent=21).calculate(prices_basic)
    print(f"  Momentum:   {momentum_scores.dropna().shape[0]} tickers")

    quality_scores = QualityFactor(lookback=252).calculate(prices_basic)
    print(f"  Quality:    {quality_scores.dropna().shape[0]} tickers")

    # Value factor needs expense ratios - use synthetic if unavailable
    fundamentals_path = RAW_DIR / "fundamentals.csv"
    if fundamentals_path.exists():
        fund_df = pd.read_csv(fundamentals_path)
        expense_ratios = fund_df.set_index("ticker")["expense_ratio"].dropna()
        expense_ratios = expense_ratios.reindex(prices_basic.columns).fillna(expense_ratios.median())
    else:
        expense_ratios = pd.Series(
            np.random.uniform(0.0005, 0.01, len(prices_basic.columns)),
            index=prices_basic.columns,
        )

    value_scores = SimplifiedValueFactor().calculate(prices_basic, expense_ratios)
    print(f"  Value:      {value_scores.dropna().shape[0]} tickers")

    volatility_scores = VolatilityFactor(lookback=60).calculate(prices_basic)
    print(f"  Volatility: {volatility_scores.dropna().shape[0]} tickers")

    # Integrate
    factor_df = pd.DataFrame(
        {
            "momentum": momentum_scores,
            "quality": quality_scores,
            "value": value_scores,
            "volatility": volatility_scores,
        }
    )

    integrator = FactorIntegrator(factor_weights=FACTOR_WEIGHTS)
    combined_scores = integrator.integrate(factor_df)

    # Save
    combined_scores.to_frame("integrated_score").to_parquet(factor_scores_path)
    print(f"\nIntegrated scores: {len(combined_scores)} tickers")

# Show top 30
if 'combined_scores' in dir():
    display(Markdown("### Top 30 ETFs by Integrated Score"))
    top30 = combined_scores.head(30).to_frame("score")
    top30["category"] = [categories.get(t, "") for t in top30.index]
    display(top30.style.format({"score": "{:.4f}"}))

---
## Section 4: Portfolio Construction & Optimization

In [ ]:
from portfolio import (
    MeanVarianceOptimizer,
    MinVarianceOptimizer,
    RankBasedOptimizer,
    SimpleOptimizer,
)

print(f"Optimizer: {OPTIMIZER_TYPE.upper()}, Positions: {NUM_POSITIONS}")

# Use basic (non-leveraged) prices for optimization
opt_prices = prices_basic if 'prices_basic' in dir() else prices

if OPTIMIZER_TYPE == "mvo":
    optimizer = MeanVarianceOptimizer(
        num_positions=NUM_POSITIONS,
        lookback=60,
        risk_aversion=1.0,
        use_factor_scores_as_alpha=True,
        min_weight=0.03,
        max_weight=0.08,
    )
    target_weights = optimizer.optimize(combined_scores, opt_prices)
elif OPTIMIZER_TYPE == "rankbased":
    optimizer = RankBasedOptimizer(
        num_positions=NUM_POSITIONS,
        weighting_scheme="exponential",
    )
    target_weights = optimizer.optimize(combined_scores)
elif OPTIMIZER_TYPE == "minvar":
    optimizer = MinVarianceOptimizer(num_positions=NUM_POSITIONS, lookback=60)
    target_weights = optimizer.optimize(combined_scores, opt_prices)
else:
    optimizer = SimpleOptimizer(num_positions=NUM_POSITIONS)
    target_weights = optimizer.optimize(combined_scores)

# Save
target_weights.to_csv(LIVE_DIR / "target_portfolio_latest.csv", header=True)

# Display
port_info = pd.DataFrame(
    {
        "weight": target_weights,
        "score": combined_scores[target_weights.index],
        "category": [categories.get(t, "") for t in target_weights.index],
    }
).sort_values("weight", ascending=False)

display(Markdown(f"### Target Portfolio ({len(target_weights)} positions)"))
display(port_info.style.format({"weight": "{:.1%}", "score": "{:.4f}"}))

print(f"\nMax weight:  {target_weights.max():.1%}")
print(f"Min weight:  {target_weights.min():.1%}")
print(f"HHI:         {(target_weights**2).sum():.4f}")

# Expected volatility
if opt_prices is not None and len(opt_prices) > 60:
    available = [t for t in target_weights.index if t in opt_prices.columns]
    if len(available) == len(target_weights):
        ret = opt_prices[available].pct_change().dropna()
        cov = ret.cov() * 252
        w = target_weights[available].values
        port_vol = np.sqrt(w @ cov.values @ w)
        print(f"Expected vol: {port_vol:.1%}")

---
## Section 5: Backtesting & Performance Analysis

In [ ]:
from backtesting import BacktestConfig, BacktestEngine, TransactionCostModel
from portfolio import RankBasedOptimizer, StopLossManager, ThresholdRebalancer

config = BacktestConfig(
    initial_capital=1_000_000,
    rebalance_frequency="weekly",
    num_positions=NUM_POSITIONS,
    stop_loss_pct=STOP_LOSS_PCT,
    use_stop_loss=True,
    risk_free_rate=0.04,
)

cost_model = TransactionCostModel(
    commission_per_trade=0.0,
    spread_bps=2.0,
    slippage_bps=2.0,
)

# Use basic (non-leveraged) prices for backtesting
bt_prices = prices_basic if 'prices_basic' in dir() else prices

# Build factor scores over time (static scores replicated across dates)
factor_scores_bt = pd.DataFrame(
    {col: combined_scores for col in bt_prices.index}
).T
factor_scores_bt.index = bt_prices.index

bt_optimizer = RankBasedOptimizer(
    num_positions=NUM_POSITIONS,
    weighting_scheme="exponential",
)
rebalancer = ThresholdRebalancer(drift_threshold=DRIFT_THRESHOLD)
risk_manager = StopLossManager(position_stop_loss=STOP_LOSS_PCT)

print("Running backtest...")
results = BacktestEngine(config, cost_model).run(
    prices=bt_prices,
    factor_scores=factor_scores_bt,
    optimizer=bt_optimizer,
    rebalancer=rebalancer,
    risk_manager=risk_manager,
)

metrics = results["metrics"]
print("Done.\n")

display(Markdown("### Backtest Results"))
for k in ["cagr", "sharpe_ratio", "sortino_ratio", "max_drawdown",
          "volatility", "win_rate", "total_return", "num_rebalances"]:
    v = metrics.get(k, 0)
    if isinstance(v, float) and abs(v) < 10:
        print(f"  {k:<20} {v:+.2%}" if "ratio" not in k else f"  {k:<20} {v:.2f}")
    else:
        print(f"  {k:<20} {v}")

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

daily_values = results["daily_values"]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), gridspec_kw={"height_ratios": [3, 1]})

# Equity curve
axes[0].plot(daily_values.index, daily_values.values, linewidth=1.5, color="steelblue")
axes[0].set_title("Equity Curve", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Portfolio Value ($)")
axes[0].grid(True, alpha=0.3)
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x:,.0f}"))

# Drawdown
running_max = daily_values.expanding().max()
drawdown = (daily_values - running_max) / running_max
axes[1].fill_between(drawdown.index, drawdown.values, 0, alpha=0.4, color="red")
axes[1].set_title("Drawdown", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Drawdown")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Max drawdown: {drawdown.min():.1%} on {drawdown.idxmin().date()}")

---
## Section 6: Trade Recommendations

Connects to IB, pulls live positions, compares vs target, generates trade plan.
Enforces $70,000 cash reserve on all buy orders.

In [ ]:
from ib_insync import IB, Order, Stock, LimitOrder, MarketOrder

ib = IB()
ib_connected = False
try:
    ib.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID, readonly=True, timeout=10)
    print(f"Connected: {ib.managedAccounts()[0]}")
    ib_connected = True
except Exception as e:
    print(f"IB not available: {e}")
    print("Trade recommendations will be generated from target portfolio only.")

In [ ]:
if ib_connected:
    # Pull account summary
    summary = {}
    for av in ib.accountSummary():
        if av.currency == "USD":
            try:
                summary[av.tag] = float(av.value)
            except (ValueError, TypeError):
                pass

    nlv = summary.get("NetLiquidation", 0)
    cash = summary.get("TotalCashValue", 0)

    # Pull positions
    ib_positions = [
        {"ticker": p.contract.symbol, "shares": float(p.position), "avg_cost": float(p.avgCost)}
        for p in ib.positions()
    ]

    # Fetch live prices
    all_tickers_live = sorted(set(p["ticker"] for p in ib_positions) | set(target_weights.index))
    ib.reqMarketDataType(3)
    live_prices = {}
    qc = []
    for t in all_tickers_live:
        c = Stock(t, "SMART", "USD")
        try:
            ib.qualifyContracts(c)
            if c.conId:
                qc.append(c)
        except Exception:
            pass
    snaps = [(c.symbol, ib.reqMktData(c, snapshot=True)) for c in qc]
    ib.sleep(4)
    for sym, td in snaps:
        for attr in ("last", "close", "bid", "ask"):
            v = getattr(td, attr, None)
            if v is not None and v == v and v > 0:
                live_prices[sym] = float(v)
                break
        ib.cancelMktData(td.contract)
    ib.reqMarketDataType(1)

    # Account overview
    deployable = max(0, cash - CASH_RESERVE)
    display(Markdown(f"""
### Account Overview
| Metric | Value |
|--------|-------|
| Net Liquidation | ${nlv:,.2f} |
| Cash | ${cash:,.2f} |
| Cash Reserve | ${CASH_RESERVE:,} |
| Deployable Cash | ${deployable:,.2f} |
| Positions | {len(ib_positions)} |
"""))
    print(f"Got {len(live_prices)} live prices.")
else:
    print("Skipping - IB not connected.")

In [ ]:
trades = []

if ib_connected:
    # Build position data
    targets = target_weights.to_dict()
    ib_map = {}
    pos_data = []

    for pos in ib_positions:
        t = pos["ticker"]
        if pos["shares"] == 0:
            continue
        price = live_prices.get(t, pos["avg_cost"])
        mv = pos["shares"] * price
        tw = targets.get(t, 0)
        aw = mv / nlv if nlv else 0
        ib_map[t] = pos
        pos_data.append({
            "ticker": t, "shares": pos["shares"],
            "price": price, "mkt_value": mv,
            "target_w": tw, "actual_w": aw,
            "drift": aw - tw, "in_target": t in targets,
        })

    # 1. Sell non-targets
    special = {"IBKR"}
    for p in pos_data:
        if not p["in_target"] and p["ticker"] not in special:
            if p["shares"] > 0:
                trades.append({"action": "SELL", "ticker": p["ticker"],
                               "shares": int(abs(p["shares"])), "price": round(p["price"], 2),
                               "est_value": round(abs(p["mkt_value"]), 2),
                               "reason": "Not in strategy target", "instruction": "APPROVE"})
            elif p["shares"] < 0:
                trades.append({"action": "BUY_TO_COVER", "ticker": p["ticker"],
                               "shares": int(abs(p["shares"])), "price": round(p["price"], 2),
                               "est_value": round(abs(p["mkt_value"]), 2),
                               "reason": "Close short position", "instruction": "APPROVE"})

    # Cash reserve calculation
    sell_proceeds = sum(t["est_value"] for t in trades if t["action"] == "SELL")
    cover_cost = sum(t["est_value"] for t in trades if t["action"] == "BUY_TO_COVER")
    available = max(0, cash + sell_proceeds - cover_cost - CASH_RESERVE)
    print(f"Available for buys: ${available:,.0f}")

    # 2. Buy candidates
    buy_candidates = []
    for ticker, tw in targets.items():
        if ticker not in ib_map:
            price = live_prices.get(ticker)
            if price and price > 0:
                sh = int(tw * nlv / price)
                if sh > 0:
                    buy_candidates.append({"action": "BUY", "ticker": ticker,
                                           "shares": sh, "price": round(price, 2),
                                           "est_value": round(sh * price, 2),
                                           "reason": f"New position (target {tw*100:.0f}%)",
                                           "instruction": "APPROVE"})

    for p in pos_data:
        if p["target_w"] > 0 and abs(p["drift"]) > 0.03:
            diff = p["target_w"] * nlv - p["mkt_value"]
            ds = int(abs(diff) / p["price"])
            if ds > 0:
                if diff > 0:
                    buy_candidates.append({"action": "BUY", "ticker": p["ticker"],
                                           "shares": ds, "price": round(p["price"], 2),
                                           "est_value": round(ds * p["price"], 2),
                                           "reason": f"Rebalance (drift {p['drift']*100:+.1f}%)",
                                           "instruction": "APPROVE"})
                else:
                    trades.append({"action": "SELL", "ticker": p["ticker"],
                                   "shares": ds, "price": round(p["price"], 2),
                                   "est_value": round(ds * p["price"], 2),
                                   "reason": f"Rebalance (drift {p['drift']*100:+.1f}%)",
                                   "instruction": "APPROVE"})

    # Apply cash reserve cap
    running_spend = 0
    for bc in buy_candidates:
        if running_spend + bc["est_value"] <= available:
            trades.append(bc)
            running_spend += bc["est_value"]
        else:
            remaining = available - running_spend
            if remaining > bc["price"]:
                reduced = int(remaining / bc["price"])
                if reduced > 0:
                    bc["shares"] = reduced
                    bc["est_value"] = round(reduced * bc["price"], 2)
                    bc["reason"] += f" [capped by ${CASH_RESERVE:,} reserve]"
                    trades.append(bc)
                    running_spend += bc["est_value"]

    trades.sort(key=lambda t: (0 if "SELL" in t["action"] else 1, t["ticker"]))

    # Archive and save
    if TRADE_PLAN_FILE.exists():
        mtime = datetime.fromtimestamp(TRADE_PLAN_FILE.stat().st_mtime)
        dest = ARCHIVE_DIR / f"trade_plan_{mtime.strftime('%Y%m%d_%H%M%S')}.csv"
        shutil.copy2(TRADE_PLAN_FILE, dest)
        print(f"Archived previous plan: {dest.name}")

    ts = datetime.now().strftime("%Y-%m-%d %H:%M")
    with open(TRADE_PLAN_FILE, "w", newline="") as f:
        f.write(f"# TRADE PLAN - Generated {ts}\n#\n")
        w = csv.DictWriter(f, fieldnames=["action", "ticker", "shares", "price", "est_value", "reason", "instruction"])
        w.writeheader()
        for t in trades:
            w.writerow(t)

    total_buys = sum(t["est_value"] for t in trades if "BUY" in t["action"])
    total_sells = sum(t["est_value"] for t in trades if t["action"] == "SELL")

    display(Markdown(f"### Trade Plan ({len(trades)} trades)"))
    display(pd.DataFrame(trades))
    print(f"\nBuys: ${total_buys:,.0f}  |  Sells: ${total_sells:,.0f}  |  "
          f"Cash after: ${cash + total_sells - total_buys:,.0f} (reserve: ${CASH_RESERVE:,})")
    print(f"Saved: {TRADE_PLAN_FILE}")
else:
    print("Skipping - IB not connected.")

---
## Section 7: Edit Instructions & Claude Interpretation

Edit `custom_instructions` below. Claude interprets natural language.

In [ ]:
# ============================================================
# EDIT YOUR INSTRUCTIONS HERE
# Format: "TICKER": "instruction"
# Everything not listed stays as APPROVE.
#
# Examples:
#   "BND": "SKIP",
#   "DIM": "reduce to 30 shares",
#   "EWU": "change to limit order at $45",
# ============================================================

custom_instructions = {
    # "BND": "SKIP",
}

# ============================================================
# Apply
for t in trades:
    if t["ticker"] in custom_instructions:
        t["instruction"] = custom_instructions[t["ticker"]]

if trades:
    display(Markdown("### Updated Trade Plan"))
    display(pd.DataFrame(trades))

In [ ]:
import re


def interpret_trades(trades):
    """Interpret instructions using Claude API or keyword fallback."""
    result = []
    custom = [t for t in trades if t["instruction"].upper() not in ("APPROVE", "SKIP", "")]

    claude_map = {}
    if custom:
        api_key = os.environ.get("ANTHROPIC_API_KEY")
        if api_key:
            try:
                import anthropic

                client = anthropic.Anthropic(api_key=api_key)
                trade_lines = [
                    f"  - {t['action']} {t['shares']} shares of {t['ticker']} @ ${t['price']:.2f} | "
                    f"User instruction: \"{t['instruction']}\""
                    for t in custom
                ]
                prompt = (
                    "You are a trade execution assistant. Interpret each instruction into "
                    "concrete trade parameters.\n\nReturn a JSON array of objects with: "
                    '"ticker", "action" (BUY/SELL/BUY_TO_COVER/SKIP), "order_type" '
                    '(MARKET/LIMIT/STOP), "shares" (int), "limit_price" (number or null), '
                    '"stop_price" (number or null), "note" (string).\n\n'
                    f"Trades:\n{chr(10).join(trade_lines)}\n\nReturn ONLY the JSON array."
                )
                print("Sending to Claude...")
                msg = client.messages.create(
                    model="claude-sonnet-4-5-20250929",
                    max_tokens=1024,
                    messages=[{"role": "user", "content": prompt}],
                )
                text = msg.content[0].text.strip()
                if text.startswith("```"):
                    text = text.split("```")[1]
                    if text.startswith("json"):
                        text = text[4:]
                interpreted = json.loads(text)
                claude_map = {i["ticker"]: i for i in interpreted}
                print(f"Claude interpreted {len(interpreted)} trade(s).")
            except Exception as e:
                print(f"Claude failed: {e}. Using keyword fallback.")
        else:
            print("No ANTHROPIC_API_KEY. Using keyword fallback.")

    for t in trades:
        instr = t["instruction"].upper()
        if instr in ("SKIP", ""):
            continue
        if instr == "APPROVE":
            result.append({"ticker": t["ticker"], "action": t["action"],
                           "order_type": "MARKET", "shares": t["shares"],
                           "limit_price": None, "stop_price": None, "note": "Approved"})
        elif t["ticker"] in claude_map:
            ci = claude_map[t["ticker"]]
            if ci["action"] != "SKIP":
                result.append(ci)
            else:
                print(f"  Skipping {t['ticker']}: {ci.get('note', '')}")
        else:
            il = t["instruction"].lower()
            if "reduce" in il:
                m = re.search(r"(\d+)\s*shares?", il)
                if m:
                    result.append({"ticker": t["ticker"], "action": t["action"],
                                   "order_type": "MARKET", "shares": int(m.group(1)),
                                   "limit_price": None, "stop_price": None,
                                   "note": f"Reduced to {m.group(1)} shares"})
                    continue
            if "limit" in il:
                m = re.search(r"\$?([\d.]+)", il)
                if m:
                    result.append({"ticker": t["ticker"], "action": t["action"],
                                   "order_type": "LIMIT", "shares": t["shares"],
                                   "limit_price": float(m.group(1)), "stop_price": None,
                                   "note": f"Limit @ ${m.group(1)}"})
                    continue
            print(f"  Cannot parse '{t['instruction']}' for {t['ticker']} - skipping")

    return result


final_trades = interpret_trades(trades) if trades else []

if final_trades:
    display(Markdown(f"### Execution Plan ({len(final_trades)} trades)"))
    display(pd.DataFrame(final_trades))
else:
    print("No trades to execute.")

---
## Section 8: Execute Trades

**This places REAL orders on your IB account.**

Set `CONFIRM = True` and run.

In [ ]:
# ============================================================
# SET TO True TO EXECUTE TRADES
# ============================================================
CONFIRM = False
# ============================================================

if not CONFIRM:
    print("Execution not confirmed. Set CONFIRM = True to execute.")
elif not final_trades:
    print("No trades to execute.")
elif not ib_connected:
    print("IB not connected. Cannot execute.")
else:
    # Reconnect in read-write mode
    ib.disconnect()
    ib_rw = IB()
    ib_rw.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID + 10, readonly=False, timeout=10)
    print(f"Connected for trading: {ib_rw.managedAccounts()[0]}\n")

    exec_results = []
    for t in final_trades:
        if t["action"] == "SKIP":
            continue

        ticker = t["ticker"]
        ib_action = "BUY" if t["action"] in ("BUY", "BUY_TO_COVER") else "SELL"
        shares = t["shares"]
        order_type = t.get("order_type", "MARKET")

        contract = Stock(ticker, "SMART", "USD")
        try:
            ib_rw.qualifyContracts(contract)
        except Exception as e:
            print(f"  {ticker}: FAILED - {e}")
            exec_results.append({"ticker": ticker, "status": "FAILED", "message": str(e)})
            continue

        if order_type == "LIMIT" and t.get("limit_price"):
            order = LimitOrder(ib_action, shares, t["limit_price"])
        else:
            order = MarketOrder(ib_action, shares)

        print(f"  {ib_action} {shares} {ticker} ({order_type})...", end="")
        trade_obj = ib_rw.placeOrder(contract, order)
        ib_rw.sleep(2)

        status = trade_obj.orderStatus.status
        fill = trade_obj.orderStatus.avgFillPrice
        print(f" {status}" + (f" @ ${fill:.2f}" if fill else ""))

        exec_results.append({"ticker": ticker, "status": status,
                             "order_id": trade_obj.order.orderId, "fill_price": fill})

        # Trailing stop for BUY fills
        if (t["action"] == "BUY"
                and status in ("Filled", "Submitted", "PreSubmitted")
                and fill and fill > 0):
            ts_order = Order()
            ts_order.action = "SELL"
            ts_order.totalQuantity = shares
            ts_order.orderType = "TRAIL"
            ts_order.trailingPercent = TRAILING_STOP_PCT
            ts_order.tif = "GTC"
            ts_trade = ib_rw.placeOrder(contract, ts_order)
            ib_rw.sleep(1)
            init_stop = fill * (1 - TRAILING_STOP_PCT / 100)
            print(f"    Trailing stop {TRAILING_STOP_PCT}% (~${init_stop:.2f}): {ts_trade.orderStatus.status}")

    # Log
    log_file = LIVE_DIR / "execution_log.csv"
    file_exists = log_file.exists()
    log_ts = datetime.now().isoformat()
    with open(log_file, "a", newline="") as f:
        w = csv.writer(f)
        if not file_exists:
            w.writerow(["timestamp", "ticker", "status", "order_id", "fill_price"])
        for r in exec_results:
            w.writerow([log_ts, r.get("ticker"), r.get("status"),
                        r.get("order_id", ""), r.get("fill_price", "")])

    # Summary
    filled = sum(1 for r in exec_results if r["status"] in ("Filled", "Submitted", "PreSubmitted"))
    failed = sum(1 for r in exec_results if r["status"] in ("ERROR", "FAILED"))
    display(Markdown("### Execution Summary"))
    print(f"Executed: {filled}  |  Failed: {failed}  |  Total: {len(exec_results)}")
    display(pd.DataFrame(exec_results))

    ib_rw.disconnect()
    print("\nDisconnected.")

---
## Cleanup

In [ ]:
if ib.isConnected():
    ib.disconnect()
    print("Disconnected from IB Gateway.")
else:
    print("Already disconnected.")